# Описание проекта

По лору, заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила нас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике.

Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса.

## Импортируем нужные библиотеки, подключимся к базе данных и посмотрим на имеющиеся данные

In [ ]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [ ]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [ ]:
query = """
SELECT
user_id,
lt_day,
IIF(lt_day <= 365, 'new', 'used') 
AS is_new,
age,
CASE WHEN gender_segment = 1
        THEN 'female'
    WHEN gender_segment = 0
        THEN 'male'
END AS 'gender_segment',
os_name,
cpe_type_name,
country,
city,
SUBSTR(ags.title, 4) 
AS age_segment,
SUBSTR(ts.title, 4) 
AS traffic_segment,
SUBSTR(ls.title, 4) 
AS lifetime_segment,
nps_score,
CASE WHEN nps_score >= 9
        THEN 'promoters'
    WHEN nps_score >= 7
        THEN 'passives'
    ELSE 'detractors'
END AS nps_group
FROM user INNER JOIN location AS loc ON user.location_id = loc.location_id
INNER JOIN age_segment AS ags ON user.age_gr_id = ags.age_gr_id
INNER JOIN traffic_segment AS ts ON user.tr_gr_id = ts.tr_gr_id
INNER JOIN lifetime_segment AS ls ON user.lt_gr_id = ls.lt_gr_id;
"""

In [ ]:
df = pd.read_sql(query, engine)
df.sample(20)

In [ ]:
df.to_csv('df_telecomm.csv', index=False)

## Укажите ссылку на дашборд на сайте Tableau Public:

https://public.tableau.com/views/Dashboard_Proj_2/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

## Укажите ссылку на pdf-файл с презентацией:

https://disk.yandex.ru/i/qtj9tFUERdRVWA